# Задание 3- Pandas

In [1]:
#Импортируем необходимые библиотеки
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

## Рассмотрим вспомогательные таблицы

### Состав данных lectures.csv : 

**lecture_id**: внешний ключ для столбца content_type_id в train.csv, если тип контента - лекция (1).  
**part**: раздел, к которому относится лекция.  
**tag**: один тег для каждой лекции. Значения тегов не представлены, но их достаточно для кластеризации лекций.  
**type_of**: краткое описание основной цели лекции.

In [2]:
# Получаем таблицу
lectures_df = pd.read_csv('data\lectures.csv', encoding = 'utf-8')
lectures_df.shape

(418, 4)

In [3]:
#Проверяем заголовки
lectures_df.head()

,lecture_id,tag,part,type_of
0,89,159,5,concept
1,100,70,1,concept
2,185,45,6,concept
3,192,79,5,solving question
4,317,156,5,solving question


In [4]:
# Получаем распределение значений в таблице
lectures_df.describe() 

,lecture_id,tag,part
count,418.000000,418.000000,418.000000
mean,16983.401914,94.480861,4.267943
std,9426.164660,53.586487,1.872424
min,89.000000,0.000000,1.000000
25%,9026.250000,50.250000,2.000000
50%,17161.500000,94.500000,5.000000
75%,24906.250000,140.000000,6.000000
max,32736.000000,187.000000,7.000000


In [5]:
# Проверяем пропуски
lectures_df.isna().sum() 

lecture_id    0
tag           0
part          0
type_of       0
dtype: int64

##### Пропусков нет

In [6]:
#Проверяем дубликаты
lectures_df.duplicated().sum()

0

##### Пропусков нет

In [7]:
#Проверяем типы данных
lectures_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 4 columns):
lecture_id    418 non-null int64
tag           418 non-null int64
part          418 non-null int64
type_of       418 non-null object
dtypes: int64(3), object(1)
memory usage: 13.1+ KB


In [8]:
#Выводим распеределение внешних ключей по различным разделам лекций
lectures_df[['type_of', 'part', 'lecture_id']].groupby(['type_of','part'], as_index = False).count().sort_values(by = ['type_of', 'part'])

,type_of,part,lecture_id
0,concept,1,20
1,concept,2,18
2,concept,3,7
3,concept,4,19
4,concept,5,72
5,concept,6,63
6,concept,7,23
7,intention,2,7
8,solving question,1,34
9,solving question,2,31


### Состав данных questions.csv : 

**question_id**: внешний ключ для столбца content_type_id в train.csv, если тип контента - вопрос (0).  
**bundle_id**: идентификатор совместного набора вопросов.  
**correct_answer**: правильный ответ на вопрос. Можно сравнить со столбцом user_answer в train.csv, чтобы проверить, верно ли ответил студент.  
**part**: соответствующий раздел теста TOEIC.  
**tags**: один или несколько подробных тегов для каждого вопроса. Значения тегов не представлены, но их достаточно для кластеризации вопросов.

In [15]:
# Получаем таблицу
questions_df = pd.read_csv('data/questions.csv') 
questions_df.shape

(13523, 5)

In [16]:
# Проверяем заголовки
questions_df.head() 

,question_id,bundle_id,correct_answer,part,tags
0,0,0,0,1,51 131 162 38
1,1,1,1,1,131 36 81
2,2,2,0,1,131 101 162 92
3,3,3,0,1,131 149 162 29
4,4,4,3,1,131 5 162 38


In [17]:
# Получаем распределение значений в таблице
questions_df.describe()

,question_id,bundle_id,correct_answer,part
count,13523.00000,13523.000000,13523.000000,13523.000000
mean,6761.00000,6760.510907,1.455298,4.264956
std,3903.89818,3903.857783,1.149707,1.652553
min,0.00000,0.000000,0.000000,1.000000
25%,3380.50000,3379.500000,0.000000,3.000000
50%,6761.00000,6761.000000,1.000000,5.000000
75%,10141.50000,10140.000000,3.000000,5.000000
max,13522.00000,13522.000000,3.000000,7.000000


In [18]:
# Проверяем пропуски
questions_df.isna().sum() 

question_id       0
bundle_id         0
correct_answer    0
part              0
tags              1
dtype: int64

##### Нашли 1 пропуск, получим информацию о нем

In [20]:
questions_df[questions_df.tags.isna()]

,question_id,bundle_id,correct_answer,part,tags
10033,10033,10033,2,6,NaN


In [21]:
#Проверяем дубликаты
questions_df.duplicated().sum() 

0

##### дубликатов нет

In [22]:
#Проверяем типы данных
questions_df.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13523 entries, 0 to 13522
Data columns (total 5 columns):
question_id       13523 non-null int64
bundle_id         13523 non-null int64
correct_answer    13523 non-null int64
part              13523 non-null int64
tags              13522 non-null object
dtypes: int64(4), object(1)
memory usage: 528.3+ KB


## Рассмотрим основную таблицу

### Состав данных train.csv:

**row_id**: идентификатор строки.  
**timestamp**: время в миллисекундах между этим взаимодействием со студентом и завершением первого события от данного студента.  
**user_id**: идентификатор студента.  
**content_id**: идентификатор взаимодействия со студентом.  
**content_type_id**: идентификатор типа контента. 0, если контентом был заданный студенту вопрос, 1 - просмотр студентом лекции.  
**task_container_id**: идентификатор набора вопросов или лекций. Например, студент может увидеть три вопроса подряд, прежде чем увидит пояснения к какому-либо из них. Все три вопроса будут иметь общий идентификатор набора.  
**user_answer**: ответ студента на вопрос, если они есть. Для лекций считать -1 как null.  
**answered_correctly**: был ли ответ студента на вопрос правильным. Для лекций считать -1 как null.  
**prior_question_elapsed_time**: среднее время в миллисекундах, затрачиваемое студентом для ответа на каждый вопрос из предыдущего набора вопросов, без учёта промежуточных лекций. Имеет значение null для 1-го набора вопросов или лекции.  
**prior_question_had_explanation**: увидел ли студент объяснение и правильный ответ после ответа на предыдущий набор вопросов, без учёта промежуточных лекций. Это значение является общим для одного набора вопросов и имеет значение null для 1-го набора вопросов или лекции. Как правило, первые несколько вопросов, увиденные студентом, были частью вводного диагностического теста и не получили никакой обратной связи.

In [23]:
#Получаем таблицу
df = pd.read_csv('data/train.csv', encoding = 'utf-8')
df.shape

(101230332, 10)

In [24]:
# Проверяем заголовки
df.head()

,row_id,timestamp,user_id,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation
0,0,0,115,5692,0,1,3,1,NaN,NaN
1,1,56943,115,5716,0,2,2,1,37000.0,False
2,2,118363,115,128,0,0,0,1,55000.0,False
3,3,131167,115,7860,0,3,0,1,19000.0,False
4,4,137965,115,7922,0,4,1,1,11000.0,False


In [26]:
# Получаем распределение значений в таблице
df.describe()

,row_id,timestamp,user_id,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time
count,1.012303e+08,1.012303e+08,1.012303e+08,1.012303e+08,1.012303e+08,1.012303e+08,1.012303e+08,1.012303e+08,9.887879e+07
mean,5.061517e+07,7.703644e+09,1.076732e+09,5.219605e+03,1.935222e-02,9.040624e+02,1.376123e+00,6.251644e-01,2.542381e+04
std,2.922268e+07,1.159266e+10,6.197163e+08,3.866359e+03,1.377596e-01,1.358302e+03,1.192896e+00,5.225307e-01,1.994815e+04
min,0.000000e+00,0.000000e+00,1.150000e+02,0.000000e+00,0.000000e+00,0.000000e+00,-1.000000e+00,-1.000000e+00,0.000000e+00
25%,2.530758e+07,5.243436e+08,5.408116e+08,2.063000e+03,0.000000e+00,1.040000e+02,0.000000e+00,0.000000e+00,1.600000e+04
50%,5.061517e+07,2.674234e+09,1.071781e+09,5.026000e+03,0.000000e+00,3.820000e+02,1.000000e+00,1.000000e+00,2.100000e+04
75%,7.592275e+07,9.924551e+09,1.615742e+09,7.425000e+03,0.000000e+00,1.094000e+03,3.000000e+00,1.000000e+00,2.966600e+04
max,1.012303e+08,8.742577e+10,2.147483e+09,3.273600e+04,1.000000e+00,9.999000e+03,3.000000e+00,1.000000e+00,3.000000e+05


In [25]:
#Проверяем типы данных
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101230332 entries, 0 to 101230331
Data columns (total 10 columns):
row_id                            int64
timestamp                         int64
user_id                           int64
content_id                        int64
content_type_id                   int64
task_container_id                 int64
user_answer                       int64
answered_correctly                int64
prior_question_elapsed_time       float64
prior_question_had_explanation    object
dtypes: float64(1), int64(8), object(1)
memory usage: 7.5+ GB


**Из представленных данных можно сделать следующие выводы:**
1. столбец **row_id** не несет полезной нагрузки- его можно удалить
2. датафрейм занимает 7.5+ GB памяти
3. попробуем уменьшить объем для облегчения работы с ним

In [27]:
# Удаляем столбец row_id и заменяем данные
df = df.drop('row_id', axis = 1)
df['timestamp'] = df['timestamp'].astype('uint64')
df['user_id'] = df['user_id'].astype('uint32')
df['content_id'] = df['content_id'].astype('uint16')
df['content_type_id'] = df['content_type_id'].astype('int8')
df['task_container_id'] = df['task_container_id'].astype('uint16')
df['user_answer'] = df['user_answer'].astype('int8')
df['answered_correctly'] = df['answered_correctly'].astype('int8')
df['prior_question_elapsed_time'] = df['prior_question_elapsed_time'].astype('float32')

In [28]:
# Повторно проверяем типы и размер таблицы
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101230332 entries, 0 to 101230331
Data columns (total 9 columns):
timestamp                         uint64
user_id                           uint32
content_id                        uint16
content_type_id                   int8
task_container_id                 uint16
user_answer                       int8
answered_correctly                int8
prior_question_elapsed_time       float32
prior_question_had_explanation    object
dtypes: float32(1), int8(3), object(1), uint16(2), uint32(1), uint64(1)
memory usage: 2.9+ GB


После удаления лишнего столбца и замены типов данных размер фрейма уменьшился до 2,9 с такими данными работать будет проще

### Проведём очистку данных:

In [29]:
# Ищем пропуски
df.isna().sum()

timestamp                               0
user_id                                 0
content_id                              0
content_type_id                         0
task_container_id                       0
user_answer                             0
answered_correctly                      0
prior_question_elapsed_time       2351538
prior_question_had_explanation     392506
dtype: int64

##### В столбце времени ответа  2351538 пропуска в столбце просмотра объяснений и правильных ответов 392506 

Продолжить отсюда

In [30]:
# Проверяем пропуски в столбце времени ответа на каждый вопрос
df[df.prior_question_elapsed_time.isna()]

,timestamp,user_id,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation
0,0,115,5692,0,1,3,1,NaN,NaN
46,0,124,7900,0,0,0,1,NaN,NaN
76,0,2746,5273,0,0,1,0,NaN,NaN
89,653762,2746,6808,1,14,-1,-1,NaN,False
96,0,5382,5000,0,0,0,1,NaN,NaN
117,10183847,5382,16736,1,21,-1,-1,NaN,False
212,1424348597,5382,30207,1,104,-1,-1,NaN,False
216,1425557777,5382,18545,1,121,-1,-1,NaN,False
224,0,8623,3915,0,0,3,1,NaN,NaN
295,405813029,8623,10540,1,59,-1,-1,NaN,False


In [31]:
# Проверим пропуски в столбце просмотра объяснений и правильных ответов
df[df.prior_question_had_explanation.isna()]

,timestamp,user_id,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation
0,0,115,5692,0,1,3,1,NaN,NaN
46,0,124,7900,0,0,0,1,NaN,NaN
76,0,2746,5273,0,0,1,0,NaN,NaN
96,0,5382,5000,0,0,0,1,NaN,NaN
224,0,8623,3915,0,0,3,1,NaN,NaN
336,0,8701,3901,0,0,2,1,NaN,NaN
353,0,12741,5145,0,0,3,0,NaN,NaN
624,0,13134,3926,0,0,3,1,NaN,NaN
1874,0,24418,7900,0,0,2,0,NaN,NaN
8338,0,24600,7900,0,0,0,1,NaN,NaN


In [30]:
# Поиск дубликатов
df.duplicated().sum()

0

In [35]:
#Найдем уникальных студентов 
df.agg({'user_id': pd.Series.nunique}) 

user_id    393656
dtype: int64

In [37]:
# сделаем групироваку количество ответов, правильных ответов и добавим их в датафрейм un_df
un_df = df[['user_id', 'user_answer', 'answered_correctly']].groupby(['user_id'], as_index = False).sum()

In [38]:
#посмотрим, что получилось
un_df.describe()

,user_id,user_answer,answered_correctly
count,3.936560e+05,393656.000000,393656.000000
mean,1.076358e+09,353.875889,160.763700
std,6.201319e+08,1038.467530,507.219402
min,1.150000e+02,0.000000,-11.000000
25%,5.387596e+08,37.000000,11.000000
50%,1.077717e+09,60.000000,23.000000
75%,1.613533e+09,214.000000,92.000000
max,2.147483e+09,25162.000000,14229.000000


In [39]:
# добавим столбец успеваемость
un_df['ratio'] = un_df.answered_correctly / un_df.user_answer

In [40]:
#посмотрим, что получилось
un_df.describe()

,user_id,user_answer,answered_correctly,ratio
count,3.936560e+05,393656.000000,393656.000000,3.936390e+05
mean,1.076358e+09,353.875889,160.763700,inf
std,6.201319e+08,1038.467530,507.219402,NaN
min,1.150000e+02,0.000000,-11.000000,-1.929825e-01
25%,5.387596e+08,37.000000,11.000000,3.000000e-01
50%,1.077717e+09,60.000000,23.000000,3.983402e-01
75%,1.613533e+09,214.000000,92.000000,4.759825e-01
max,2.147483e+09,25162.000000,14229.000000,inf


In [41]:
### Вычисляем медианное значение количества ответов студентов
un_med = un_df.user_answer.median()
print(int(un_med))

60


In [42]:
### Выведем распределение количества ответов студентов, которое которое больше 75%
print(un_df.query('user_answer > 151')['ratio'].median())

0.45432715062559853


In [43]:
#Выведем распределение количества ответов студентов, которое больше 50
print(un_df.query('user_answer > 50')['ratio'].median())

0.42857142857142855


In [44]:
#Выведем распределение количества ответов студентов, которое меньше 50
print(un_df.query('0 < user_answer > 50')['ratio'].median())


0.42857142857142855


In [47]:
# Сделаем групперовку кол-во ответов, кол-во правильных ответов и время ответа на каждый вопрос студентов
corr = df[['user_id', 'user_answer', 'answered_correctly', 'timestamp', \
    'prior_question_elapsed_time']].groupby(['user_id'], as_index = False).sum()

In [48]:
# Добавляем столбец timestamp соотношения времени между взаимодействием со студентом и завершением первого события от него к общему кол-ву
corr['timestamp'] = corr['timestamp'] / corr['user_answer']

In [49]:
# Добавляем столбец elapsed_time соотношения среднего времени ответа на каждый вопрос к общему кол-ву ответов студентов
corr['elapsed_time'] = corr_look['prior_question_elapsed_time'] / corr_look['user_answer']

In [50]:
# Добавляем столбец ratio успеваемости (соотношения кол-ва правильных ответов к общему кол-ву ответов студентов)
corr_look['ratio'] = corr.answered_correctly / corr.user_answer

In [54]:
#Проверяем, что получилось
corr.describe()

,user_answer,elapsed_time
count,393656.000000,3.936290e+05
mean,353.875889,inf
std,1038.467530,NaN
min,0.000000,0.000000e+00
25%,37.000000,1.317647e+04
50%,60.000000,1.596856e+04
75%,214.000000,1.949323e+04
max,25162.000000,inf


Возможные выводы
- количество ответов студентов, большее медианного значения количества ответов, равного 50
- соотношения количества правильных ответов и среднего времени ответа к общему количеству ответов студентов - слабо свзяно